In [4]:
import cv2
import numpy as np
keep_percent = 0.75
cnv = 1.4
thresh = 125.0
nfeat = 2**15 - 4
widen = 2500
# Load the images

def rot_matrix(f):
    return np.array([[np.cos(f), -np.sin(f)], [np.sin(f), np.cos(f)]])

def aff_matrix(s, t, mx):
    return np.array([[s * mx[0][0], s * mx[0][1], t[0]], [s * mx[1][0], s * mx[1][1], t[1]]])

def read_img(n):
    s1 = '2023_09_01_SonyRX1RM2_g201b20538_f001_'
    s3 = '.JPG'
    s = str(n)
    l = len(s)
    s2 = '0' * (4-l) + s
    return cv2.imread('./folder/' + s1 + s2 + s3)

def crop_img(image_1):
    mask = np.any(image_1 != 0, axis=-1)
    coords = np.argwhere(mask)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0) + 1
    return y_min, x_min, image_1[y_min:y_max, x_min:x_max]
    
def image_get_rot(image_base, image_fit):
     
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image_base, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image_fit, cv2.COLOR_BGR2GRAY)
    
    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(nfeatures = nfeat)
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
 
    # Match descriptors.
    matches = bf.match(descriptors1, descriptors2)
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    keep = int(len(matches) * keep_percent)
    matches = matches[:keep]
    

    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    print('bijection part of transform: ', len(matches), '/', nfeat, sep = '')

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find the rotation transformation matrix
    matrix, _ = cv2.estimateAffinePartial2D(points2, points1, method=cv2.RANSAC, ransacReprojThreshold = thresh)
    print('inner part of transform: ', np.count_nonzero(_ == 1), '/', nfeat, sep = '')
    print(matrix)
    sn = -matrix[0, 1]
    cn = matrix[0, 0]
    fi = np.arctan(sn/cn)
    print('angle is', fi)
    sc = cn/np.cos(fi)
    print('scale is', cn/np.cos(fi))
    tr = [matrix[0][2], matrix[1][2]]
    print('translation value is:', tr)
    return fi, sc, tr


def image_widen(image_w, w_val):
    height, width, _ = image_w.shape
    #5304x7952
    canvas_height = height + w_val
    canvas_width = width + w_val
    green_canvas = np.zeros((canvas_height, canvas_width, 4), dtype=np.uint8)
    green_canvas[:] = (0, 0, 0, 0)  # Set the canvas to green
    y_offset = (canvas_height - height) // 2
    x_offset = (canvas_width - width) // 2
    alpha_image_w = np.zeros((height, width, 4), dtype=np.uint8)
    if(image_w.shape[2] == 3):
        alpha_image_w[:,:,0:3] = image_w
        alpha_image_w[:,:,3] = 255
    else:
        alpha_image_w = image_w
    green_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = alpha_image_w
    return green_canvas

def image_overlay(image_bb, image_ol):
    h, w, _ = image_ol.shape
    print("overlay shape:", h, w)
    # Define the region of interest (ROI) on the base image
    roi = image_bb[0:h, 0:w]

    # Create a mask of the overlay image and its inverse mask
    result_gray = cv2.cvtColor(image_ol, cv2.COLOR_BGRA2GRAY)
    _, mask = cv2.threshold(result_gray, 1, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Black-out the area of the overlay in the ROI
    image1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

    # Take only the region of the overlay from the overlay image
    result_fg = cv2.bitwise_and(image_ol, image_ol, mask=mask)
    
    # Put the overlay in the ROI and modify the main image
    dst = cv2.add(image1_bg, result_fg)
    image_bb[0:h, 0:w] = dst
    return image_bb

def image_intersect(image1, image2):
    
    # Get the angle, scale and translations of the axes to fit image2 on image1
    fl, sl, tl = image_get_rot(image1, image2)

    mxl = rot_matrix(fl)
    matrix = aff_matrix(sl, tl, mxl)
    
    print(fl, sl, tl, mxl)
    print(matrix)
    
    
    # Apply the rotation transformation
    result = cv2.warpAffine(image2, matrix, (image2.shape[1], image2.shape[0]))
    cv2.imwrite('output.png', result)

    image1 = image_overlay(image1, result)
    
    # Save the cropped image
    cv2.imwrite('result_target.png', image1)
    
    print('assembled one image')
    
    return fl, sl, tl, image1

base = read_img(4)
center_h, center_w, _ = base.shape
center_h = center_h // 2
center_w = center_w // 2
print(center_h, center_w)
for i in range (5, 20):
    print('Assembling image', i, '...')
    paste = read_img(i)
    u = 4
    center_h += u * widen
    center_w += u * widen
    paste = image_widen(paste, 2 * u * widen)
    ppp = u * widen - u
    pastesq = paste[center_h  - u * widen + u: center_h + u * widen - u,center_w - u * widen + u: center_w + u * widen - u,:]
    p_center_h, p_center_w, _ = pastesq.shape
    p_center_h = p_center_h // 2
    p_center_w = p_center_w // 2
    base = image_widen(base, 2 * u * widen)
    target = base[center_h  - u * widen + u: center_h + u * widen - u,center_w - u * widen + u: center_w + u * widen - u,:]
    cv2.imwrite('target.png', target)
    fr, sr, tr, aba = image_intersect(target, pastesq)
    base[center_h - u * widen + u: center_h + u * widen - u, center_w - u * widen + u: center_w + u * widen - u] = aba
    vepp = np.array([ppp, ppp])
    delta_vepp = np.dot(rot_matrix(fr), vepp) * sr - vepp
    tr = tr + delta_vepp
    ww = int(tr[0])
    hh = int(tr[1])
    center_h += hh
    center_w += ww
    base[center_h-100:center_h+100, center_w-100:center_w+100,:] = 255
    h_min, w_min, base = crop_img(base)
    center_h -= h_min
    center_w -= w_min
    base[center_h-50:center_h+50, center_w-50:center_w+50] = [0, 0, 0, 255]
    cv2.imwrite('base.png', base)
    print('Image', i, 'has been assembled')
    
 
    

2652 3976
Assembling image 5 ...
bijection part of transform: 6699/32764
inner part of transform: 1247/32764
[[ 9.60955955e-01 -6.38023085e-02  3.07337964e+03]
 [ 6.38023085e-02  9.60955955e-01 -2.59941020e+03]]
angle is 0.06629731906590496
scale is 0.9630716913557319
translation value is: [3073.379636030175, -2599.410202723068]
0.06629731906590496 0.9630716913557319 [3073.379636030175, -2599.410202723068] [[ 0.99780314 -0.06624876]
 [ 0.06624876  0.99780314]]
[[ 9.60955955e-01 -6.38023085e-02  3.07337964e+03]
 [ 6.38023085e-02  9.60955955e-01 -2.59941020e+03]]
overlay shape: 19992 19992
assembled one image
Image 5 has been assembled
Assembling image 6 ...
bijection part of transform: 6649/32764
inner part of transform: 1265/32764
[[ 9.77426385e-01 -2.40238057e-02  1.94323836e+03]
 [ 2.40238057e-02  9.77426385e-01 -1.07013288e+03]]
angle is 0.02457368675046635
scale is 0.9777215764057662
translation value is: [1943.2383590519444, -1070.1328787149132]
0.02457368675046635 0.9777215764057

In [19]:
np.sqrt(5304**2 + 7952**2)

9558.594038874127

In [22]:
5304 * 1.4

7425.599999999999

In [30]:
import numpy as np
import cv2

# Define the transformation matrix
mat = np.array([[1, 0, 100], [0, 1, 50],[0, 0, 1]], dtype=np.float32)

# Define the point to be transformed
center_h, center_w = 200, 300
points_to_be_transformed = np.array([[[center_h, center_w]]], dtype=np.float32)

# Perform the perspective transformation
transformed_points = cv2.perspectiveTransform(points_to_be_transformed, mat)

print("Transformed Points:", transformed_points)

Transformed Points: [[[300. 350.]]]


In [1]:
import numpy as np
import cv2
angle = np.pi
s = 1.0
img = read_img(3)
h, w, _ = img.shape
print(h, w)
mat = np.array([[s * np.cos(angle), -s * np.sin(angle), w], [s * np.sin(angle), s * np.cos(angle), h]], dtype=np.float32)
result = cv2.warpAffine(img, mat, (img.shape[1], img.shape[0]))
cv2.imwrite('result_target.png', result)

NameError: name 'read_img' is not defined